## Загрузка библиотек

In [7]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 54.0 MB/s eta 0:00:00


In [8]:
!pip install --quiet -U  pyserini jsonlines fastrank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.1 MB/s eta 0:0

## Извлечение датасета

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
DATA_PATH = './data'
!mkdir data
!cp /content/drive/MyDrive/dataset/lenta-ru-filtered.csv data
!cp /content/drive/MyDrive/dataset/MSLR-WEB10K.zip data/
!unzip data/MSLR-WEB10K.zip -d data/mslr

Archive:  data/MSLR-WEB10K.zip
   creating: data/mslr/Fold1/
  inflating: data/mslr/Fold1/test.txt  
  inflating: data/mslr/Fold1/train.txt  
  inflating: data/mslr/Fold1/vali.txt  
   creating: data/mslr/Fold2/
  inflating: data/mslr/Fold2/test.txt  
  inflating: data/mslr/Fold2/train.txt  
  inflating: data/mslr/Fold2/vali.txt  
   creating: data/mslr/Fold3/
  inflating: data/mslr/Fold3/test.txt  
  inflating: data/mslr/Fold3/train.txt  
  inflating: data/mslr/Fold3/vali.txt  
   creating: data/mslr/Fold4/
  inflating: data/mslr/Fold4/test.txt  
  inflating: data/mslr/Fold4/train.txt  
  inflating: data/mslr/Fold4/vali.txt  
   creating: data/mslr/Fold5/
  inflating: data/mslr/Fold5/test.txt  
  inflating: data/mslr/Fold5/train.txt  
  inflating: data/mslr/Fold5/vali.txt  


## Подключение библиотек

In [9]:
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import scipy
import json
import re
import os
import jsonlines

from fastrank import CDataset, TrainRequest
from sklearn.datasets import load_svmlight_file

from pyserini.search.lucene import LuceneSearcher

# Поисковая система

### Загружаем данные

In [10]:
data = pd.read_csv('data/lenta-ru-filtered.csv')

In [ ]:
data

,text,tags,len,date
0,С 1 сентября на всей территории России вводитс...,Все,1654,31-08-1999
1,"По сведениям миссии ООН, передаваемым РИА ""Нов...",Все,1086,31-08-1999
2,15 представителей национал-большевистской парт...,Все,1219,31-08-1999
3,Намеченная на сегодняшний день церемония вступ...,Все,3094,31-08-1999
4,"На юге Киргизии, а именно в Баткенском и Чон-А...",Все,1354,31-08-1999
...,...,...,...,...
863280,Популярное место среди туристов в Мурманской о...,Россия,1231,11-09-2020
863281,Рейтинги от международного рейтингового агентс...,NaN,1425,11-09-2020
863282,Российские ученые нашли в Якутии новый подвид ...,События,1299,11-09-2020
863283,Для указания коронавируса как причины смерти ч...,Общество,2061,11-09-2020


### Генерация документов

In [11]:
COLLECTION_DIR = 'data/collection/lenta'
if not os.path.exists(COLLECTION_DIR):
    os.makedirs(COLLECTION_DIR)

collection = data.values

bath_size = 32768
for ind in tqdm(range(0, len(collection), bath_size)):
    files = [{'id': ind+i,
              'contents': doc[0],
              'tag': doc[1],
              'date': doc[-1]}
             for i, doc in enumerate(collection[ind:ind+bath_size])]
    with jsonlines.open(os.path.join(COLLECTION_DIR,
                                     '{}.jsonl'.format(ind)),
                        mode='w') as writer:
        writer.write_all(files)

  0%|          | 0/27 [00:00<?, ?it/s]

In [12]:
with open(os.path.join(COLLECTION_DIR,
                       '{}.jsonl'.format(ind))) as f:
  print('\n'.join(f.read().splitlines()[-5:]))

{"id": 863280, "contents": "Популярное место среди туристов в Мурманской области завалили мусором и навозом. Об этом сообщает местное издание «Мурманский вестник». По данным портала, речь идет о Лавнинских водопадах в Кольском районе региона, ставших одним из излюбленных мест отдыха северян, где можно «искупаться и сделать красивые фотографии». Местные жители жалуются, что туристы не убирают за собой мусор и пластиковые бутылки, а недавно там появились огромные лужи навоза. «На всем пути к Лавне \"аромат\" такой, аж оторопь берет... Особое негодование вызывают болотного вида лужи навоза, смердящего и отравляющего воду. А ведь лет пять назад мы набирали чайник прямо из водопада, не боялись отравиться нечистотами», — рассказала жительница района Лиза Титова. Откуда взялся навоз возле водопадов, неизвестно, но люди предполагают, что во всем могут быть виноваты фермеры, сливающие отходы на территорию курорта. В июне сообщалось, что жители английского города Бакстон подговорили фермеров заб

### Построение индекса

In [13]:
INDEX_DIR = 'data/index/lenta'
if not os.path.exists(INDEX_DIR):
    os.makedirs(INDEX_DIR)

In [14]:
!python -m pyserini.index.lucene -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 16 -input {COLLECTION_DIR} \
 -index {INDEX_DIR} -storePositions -storeDocvectors -storeRaw

2024-03-03 22:39:11,850 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-03-03 22:39:11,854 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-03-03 22:39:11,854 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-03-03 22:39:11,855 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: data/collection/lenta
2024-03-03 22:39:11,855 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-03-03 22:39:11,856 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: data/index/lenta
2024-03-03 22:39:11,856 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 16
2024-03-03 22:39:11,857 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false
2024-03-03 22:39:11,89

### Используем поисковый индекс

#### BM-25

In [23]:
searcher = LuceneSearcher(INDEX_DIR)
searcher.set_bm25()
hits = searcher.search('Поисковик', k=5)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 331463,
  "contents" : "Группа хакеров пообещала возродить Usenet-поисковик Newzbin. Соответствующее письмо было разослано зарегистрированным пользователям сервиса, пишет TorrentFreak. В письме группа хакеров, называющая себя \"Team R Dogs\", сообщила, что ей удалось получить большую часть исходного кода и баз данных ресурса. Ранее бывший редактор поисковика под ником DeepSharer сообщил в своем блоге, что, по слухам, исходный код действительно был выкраден. С помощью украденных файлов \"Team R Dogs\" надеется запустить новый поисковик под именем Newzbin 2. При этом хакер под псевдонимом Mr. White заявил: \"Мы слишком сильно любили [этот ресурс], чтобы позволить ему умереть\". Он также сообщил, что поисковик заработает в ближайшее время. В марте 2010 года британский суд постановил, что поисковик Newzbin нарушает авторские права. Основанием для принятия такого решения стала жалоба, поданная Ассоциацией кинопроизводителей (MPA). Эта организация защищает права крупнейших киносту

#### QLD

In [21]:
searcher = LuceneSearcher(INDEX_DIR)
searcher.set_qld()
hits = searcher.search('Поисковик', k=5)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 331463,
  "contents" : "Группа хакеров пообещала возродить Usenet-поисковик Newzbin. Соответствующее письмо было разослано зарегистрированным пользователям сервиса, пишет TorrentFreak. В письме группа хакеров, называющая себя \"Team R Dogs\", сообщила, что ей удалось получить большую часть исходного кода и баз данных ресурса. Ранее бывший редактор поисковика под ником DeepSharer сообщил в своем блоге, что, по слухам, исходный код действительно был выкраден. С помощью украденных файлов \"Team R Dogs\" надеется запустить новый поисковик под именем Newzbin 2. При этом хакер под псевдонимом Mr. White заявил: \"Мы слишком сильно любили [этот ресурс], чтобы позволить ему умереть\". Он также сообщил, что поисковик заработает в ближайшее время. В марте 2010 года британский суд постановил, что поисковик Newzbin нарушает авторские права. Основанием для принятия такого решения стала жалоба, поданная Ассоциацией кинопроизводителей (MPA). Эта организация защищает права крупнейших киносту

#### RM3

In [22]:
searcher = LuceneSearcher(INDEX_DIR)
searcher.set_rm3()
hits = searcher.search('Поисковик', k=5)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 338651,
  "contents" : "Китайский поисковик Baidu разрабатывает собственную мобильную ОС с открытым исходным кодом. Об этом пишет InformationWeek со ссылкой на публикации в китайских газетах. C помощью новинки Baidu рассчитывает укрепить свои позиции на рынке мобильного поиска в стране. Также она сможет составить конкуренцию мобильной ОС Google Android, которая является одной из самых популярных мобильных платформ в мире. Сообщается, что разработкой ОС занимаются бывшие сотрудники Google, которые покинули китайское представительство американской компании после того, как та прекратила осуществлять цензуру поисковых запросов в Китае. Baidu является самой популярной поисковой системой в Китае. Сообщается, что ей принадлежит более 70 процентов рынка поисковых запросов в стране. При этом по количеству мобильных поисковых запросов Baidu находится приблизительно на одном уровне с Google - на эти поисковики приходится примерно по четверти от общего числа мобильного поиска. В собстве

# Обучение ранжирования

## Загрузка датасета

In [24]:
DATA_PATH = 'data/mslr/Fold1'
(X_train, y_train, qid_train) = load_svmlight_file(
    os.path.join(DATA_PATH, 'train.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)
(X_test, y_test, qid_test) = load_svmlight_file(
    os.path.join(DATA_PATH, 'test.txt'),
    dtype=np.float32,
    zero_based=False,
    query_id=True)

train_size = 50000 # set -1 to all data
X_train = X_train.toarray()[:train_size]
y_train = y_train[:train_size]
qid_train = qid_train[:train_size]

test_size = 1000 # set -1 to all data
X_test = X_test.toarray()[:test_size]
y_test = y_test[:test_size]
qid_test = qid_test[:test_size]

In [25]:
ids = np.random.RandomState(777).randint(0, len(X_train), size=5)
(X_train[ids][:, :6], y_train[ids], qid_train[ids])

(array([[2.      , 0.      , 2.      , 0.      , 2.      , 1.      ],
        [1.      , 0.      , 1.      , 0.      , 1.      , 1.      ],
        [2.      , 0.      , 0.      , 0.      , 2.      , 0.666667],
        [4.      , 0.      , 4.      , 2.      , 4.      , 0.8     ],
        [1.      , 0.      , 1.      , 1.      , 1.      , 1.      ]],
       dtype=float32),
 array([0., 1., 0., 0., 2.]),
 array([6376, 2086, 5326, 4801, 3526]))

## Определение модели

In [26]:
train_request = TrainRequest.random_forest()
params = train_request.params
params.num_trees = 200
params.feature_sampling_rate = 0.5
params.instance_sampling_rate = 0.5

params.quiet = True
params.seed = 777

In [27]:
dataset_train = CDataset.from_numpy(X_train, y_train, qid_train)
dataset_test = CDataset.from_numpy(X_test, y_test, qid_test)

train_queries = sorted(dataset_train.queries())
test_queries = sorted(dataset_test.queries())

train = dataset_train.subsample_queries(train_queries)
test = dataset_test.subsample_queries(test_queries)

In [28]:
model = train.train_model(train_request)

In [29]:
test.evaluate(model, "NDCG@5")

{'28': 0.6333513308383866,
 '58': 0.2178022835137004,
 '88': 0.4439731298514547,
 '43': 0.49835882843314194,
 '13': 0.18459095709791615,
 '118': 0.5334141272771329,
 '103': 0.38701867902589504,
 '73': 0.15098485154804625}